<a href="https://colab.research.google.com/github/annamuyanli/MehtA-Machine-Learning-Morality-Analysis-on-Terrorism/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import requests
from pandas import json_normalize
import os
import pandas as pd
import numpy as np
import spacy
sp = spacy.load('en_core_web_sm')
import gensim
from gensim.models import Word2Vec
import gensim.downloader
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn import svm
import functools
import operator
from sklearn.metrics import accuracy_score
import requests
from sklearn.neighbors import KNeighborsClassifier
import statistics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
!pip3 install nltk
import nltk
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


#Preprocessing

##Preprocessing of our own dataset

In [ ]:
#!unzip "/content/drive/MyDrive/DHML/Final Project/El_Paso_Attack.zip" -d "/content/drive/MyDrive/DHML/Final Project/El_Paso"
#!unzip "/content/drive/MyDrive/DHML/Final Project/Vegas_Attack.zip" -d "/content/drive/MyDrive/DHML/Final Project/Vegas"

Archive:  /content/drive/MyDrive/DHML/Final Project/El_Paso_Attack.zip
replace /content/drive/MyDrive/DHML/Final Project/El_Paso/El Paso Attack/users_1157748115132309509.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/DHML/Final Project/Vegas_Attack.zip
replace /content/drive/MyDrive/DHML/Final Project/Vegas/Vegas Attack/Vegas Attack Data/users_915072754143186946.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def to_df (path_name, file_path):
  with open(file_path + "/" + path_name) as json_data:
    data=json.load(json_data)
    df=pd.DataFrame(data["tweets"])
    df = df[df["lang"]=="en"]
    df=df[['created_at','text']]
    return df

In [ ]:
def preprocessing (file_path):
  json_file = os.listdir(file_path)
  df=to_df(json_file[0], file_path)
  for i in range(1, len(json_file)):
    if json_file[i].startswith("users"):
      df2=to_df(json_file[i], file_path)
      df=df.append(df2,ignore_index=True)
  return df

In [ ]:
#df_ElPaso = preprocessing("/content/drive/MyDrive/DHML/Final Project/El_Paso/El Paso Attack")

In [ ]:
#df_ElPaso.to_csv("El_Paso_Final.csv") 
#!cp El_Paso_Final.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
#df_Vegas = preprocessing("/content/drive/MyDrive/DHML/Final Project/Vegas/Vegas Attack/Vegas Attack Data")

In [ ]:
#df_Vegas.to_csv("Vegas_Final.csv") 
#!cp Vegas_Final.csv "/content/drive/MyDrive/DHML/Final Project"

## Our training & testing data

In [ ]:
own1 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/El_Paso_Final_annotate.csv")
own2 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_Final_annotate.csv")
own1 = own1.dropna()
#print(own1)
own2 = own2.dropna()
#print(own2)
df = pd.concat([own1, own2])
df = df.drop(["Unnamed: 0"], axis = 1)

#Get features:

num of word associated, length of sentence, maximum similarity, number of hashtages, TF-IDF

###Word 2 Vec

In [ ]:
model = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


### TF-IDF

In [ ]:
def tfidf (data, data2, category):
  corpus = []
  for row in data2:
    corpus.append(row)
    #corpus.append(" ".join([word for word in row.split() if word in words.words()]))
  for row in tqdm(data):
    corpus.append(row)
    #corpus.append(" ".join([word for word in row.split() if word in words.words()]))
  vectorizer = TfidfVectorizer(token_pattern=r"\b[a-z][a-z]+\b")
  X = vectorizer.fit_transform(corpus)
  result = pd.DataFrame(X.toarray())
  #print(result)
  list_word = vectorizer.get_feature_names()
  print(list_word)
  add_column = []
  for word in tqdm(list_word):
    if word in words.words():
      index = list_word.index(word)
      add_column.append(index)
  result = result[add_column]
  #print(list_word)
  return result
  #print(X.toarray)
  #print(X.shape)

In [ ]:
Vegas = Vegas.dropna()

In [ ]:
tfidf_num = tfidf(ElPaso["text"], Vegas["text"], care1)
tfidf_num.to_csv("tfidf.csv") 
!cp tfidf.csv "/content/drive/MyDrive/DHML/Final Project"

### Get features and Labels

In [ ]:
TFIDF = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/tfidf.csv")

In [ ]:
def get_x(data, category, category_name):
  list_num = []
  list_len = []
  list_n = []
  list_hashtag = []
  for row in tqdm(data["text"].values):
    #print(row)
    num = 0
    ns = []
    hashtag = 0
    try:
      #print(row.split())
      for word in row.split():
        #print(word)
        if "#" in word:
          hashtag += 1
        if word not in model:
          continue
        for phrases in category:
          if phrases not in model:
            continue
          n = model.similarity(phrases, word)
          if n > 0.25: 
            num += 1
            ns.append(n)
    except:
      list_n.append(0)
      list_num.append(0)
      list_len.append(0)
      list_hashtag.append(0)
      continue
    if not ns:
      list_n.append(0)
    else:
      list_n.append(max(ns))
    list_num.append(num)
    list_len.append(len(row.split()))
    list_hashtag.append(hashtag)
  x = pd.DataFrame({"length": list_len, "hastages": list_hashtag, ("num "+str(category_name)): list_num, ("max_n "+str(category_name)): list_n})
  #print(x)
  return x

In [ ]:
def get_y(category):
  y = df[category]
  return y

In [ ]:
def get_x_2(data, category, category_name):
  list_num = []
  list_n = []
  for row in tqdm(data["text"].values):
    #print(row)
    num = 0
    ns = []
    hashtag = 0
    try:
      for word in row.split():
        #print(word)
        if word not in model:
          continue
        for phrases in category:
          if phrases not in model:
            continue
          n = model.similarity(phrases, word)
          if n > 0.25: 
            num += 1
            ns.append(n)
    except:
      list_num.append(0)
      list_n.append(0)
      continue
    if not ns:
      list_n.append(0)
    else:
      list_n.append(max(ns))
    list_num.append(num)
  x = pd.DataFrame({("num "+str(category_name)): list_num, ("max_n "+str(category_name)): list_n})
  #print(x)
  return x

#Data

In [ ]:
care1 = ["safe", "peace", "compassion", "empath", "sympathy", "care", "caring", "protect", "shield", "shelter", "amity", "secure", "benefit", "defense", "guard", "preserve", "harm", "suffer", "war", "wars", "warring", "fight", "violent", "hurt", "kill", "kills", "killer", "killed", "killing", "endanger", "cruel", "brutal", "abuse", "damage", "ruin", "ravage", "detriment", "crush", "attack", "annihilate", "destroy", "stomp", "abandon", "spurn", "impair", "exploit", "exploits", "exploited", "exploiting", "wound"]

fairness1 = ["fair","fairly","fairness","fairmind","fairplay","equal","justice","justness","justification","reciprocate","impartial","egalitar","rights","equity","evenness","equivalent","unbias","tolerant","equable","balance","homologous","unprejudice","reasonable","constant","honest", "unfair","unequal","bias","unjust","injust","bigotry","discriminate","disproportion","inequitable","prejudice","dishonest","unscrupulous","dissociate","preference","favoritism","segregate","exclusion","exclude"]

in_group1 = ["segregate", "together", "nation", "homeland", "family", "families", "familial", "group", "loyal", "patriot", "communal", "commune", "community", "communist", "comrade", "cadre", "collective", "joint", "unison", "unite", "fellow", "guild", "solidarity", "devote", "member", "clique", "cohort", "ally", "insider", "abandon", "foreign", "betray", "treason", "traitor", "treacherous", "disloyal", "individual", "apostasy", "apostate", "deserted", "deserter", "deserting", "deceive", "jilt ", "imposter", "miscreant", "spy", "sequester", "renegade", "terrorist", "immigrant", "racist"]

authority1 = ["obey", "obedient", "duty", "law", "lawful", "legal", "duty", "honor", "respect", "respectful", "respected", "respects", "order", "father", "mother", "mother", "mothering", "mothers", "tradition", "hierarchy", "authority", "permit", "permission","status", "rank", "leader", "class", "bourgeoisie", "caste", "position", "compliant", "command", "supremacy", "control","submit", "allegiant", "serve", "abide", "deference", "defer", "reverence ", "veneration", "comply", "betray*", "treason", "traitor", "treachery", "disloyal", "individual", "apostasy", "apostate", "deserted", "deserter","deserting", "defiant", "rebel", "dissent", "subvert", "disrespect", "disobedient*", "sedition", "agitation", "insubordination","illegal", "lawless", "insurgent", "mutinous", "defy", "dissident", "unfaithful", "alienate", "defector", "heretic","nonconformist", "oppose", "protest", "refuse", "denounce", "remonstrate", "riot", "obstruct"]

purity1 = ["piety", "pious", "purity", "pure", "clean", "sterile", "sacred", "chaste", "holy", "holiness", "saint", "wholesome","celibate", "abstention", "virgin", "virgins", "virginity", "virginal", "austerity", "integrity", "modesty", "abstinence","abstemiousness", "upright", "limpid", "unadulterated" , "maiden", "virtuous", "refined", "immaculate", "innocent","pristine", "church", "apostasy", "heretic", "disgust", "depraved", "disease", "unclean", "contagious" , "indecent", "sin", "sinful", "sinner", "sins", "sinned","sinning", "slut", "whore", "dirt", "impiety", "impious", "profane", "gross", "repulsive", "sick*", "promiscuous*", "lewd","adultery", "debauchery*", "defile", "tramp", "prostitute", "unchaste", "intemperate", "wanton", "profligate", "filth","trashy", "obscene", "lax", "tainted", "stain", "tarnish", "debased", "desecrate", "wicked", "blemish", "exploitation","pervert", "wretched"]

In [ ]:
ElPaso = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/El_Paso_Final.csv")
Vegas = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_Final.csv")

## Random Processing

In [ ]:
TFIDF_training = pd.concat([TFIDF[6204:6704], TFIDF.head(398)])

In [ ]:
TFIDF_training = TFIDF_training.drop(["Unnamed: 0"], axis = 1)

In [ ]:
TFIDF_training.reset_index(drop=True, inplace=True)

In [ ]:
TFIDF_training

,10,20,25,30,31,32,33,35,39,40,42,43,45,47,51,56,57,58,59,60,62,63,64,67,68,69,70,72,76,77,79,80,81,84,85,86,87,90,91,93,...,18381,18383,18386,18390,18391,18393,18405,18406,18407,18408,18409,18410,18413,18418,18420,18421,18424,18425,18426,18429,18433,18466,18469,18470,18477,18478,18480,18482,18484,18486,18488,18489,18490,18491,18499,18544,18551,18552,18580,18582
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.199759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df

,created_at,text,Care,Faireness,In-Group Loyalty,Respect For Authority,Purity
0,2019-08-03T20:17:40.000Z,Trump Hangs Out At His Golf Club As Americans ...,0.0,1.0,0.0,0.0,0.0
1,2019-08-03T20:11:43.000Z,"Another day, another mass shooting. This time ...",1.0,0.0,1.0,0.0,1.0
2,2019-08-03T19:39:33.000Z,Apparently Trump is too busy golfing at his Ne...,0.0,1.0,0.0,0.0,1.0
3,2019-08-03T17:59:31.000Z,"Truly heartbreaking. Stay safe, El Paso. Pleas...",1.0,0.0,0.0,0.0,1.0
4,2019-08-03T20:03:08.000Z,Scrap that. Manifesto likely legit. It was pos...,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
394,2017-10-02T17:07:19.000Z,Las Vegas shooter Stephen Paddock is son of Be...,0.0,0.0,0.0,0.0,0.0
395,2017-10-03T01:30:25.000Z,If you don't collectively blame whites for #La...,0.0,1.0,0.0,0.0,1.0
396,2017-10-03T01:48:01.000Z,"Bailey Schweitzer, 20, was a member of the Val...",1.0,1.0,0.0,0.0,1.0
397,2017-10-02T21:28:55.000Z,"My heart aches for family,friends of Rhonda Le...",1.0,0.0,0.0,0.0,1.0


In [ ]:
get_x(df, care1, "care")

,length,hastages,num care,max_n care
0,19,0,3,0.295930
1,41,0,22,0.316339
2,41,2,20,0.419239
3,21,0,14,0.369758
4,48,0,25,0.419239
...,...,...,...,...
893,19,0,3,0.382343
894,16,1,13,0.340201
895,20,1,2,0.316339
896,23,0,15,1.000000


In [ ]:
feature = pd.concat([get_x(df, care1, "care"), get_x_2(df, fairness1, "fairness"), get_x_2(df, in_group1, "loyalty"), get_x_2(df, authority1, "authority"), get_x_2(df, purity1, "purity"), TFIDF_training], axis = 1)
feature.to_csv("training_feature.csv") 
!cp training_feature.csv "/content/drive/MyDrive/DHML/Final Project"

## training/testing data feature and label

In [ ]:
feature = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/training_feature.csv")

In [ ]:
feature = feature.drop(["Unnamed: 0"], axis = 1)

In [ ]:
feature_care = feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
feature_fairness = feature.drop(["num care", "max_n care", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
feature_loyalty = feature.drop(["num fairness", "max_n fairness", "num care", "max_n care", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
feature_authority = feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num care", "max_n care", "num purity", "max_n purity"], axis = 1)
feature_purity = feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num care", "max_n care"], axis = 1)

In [ ]:
label_care = get_y("Care")

In [ ]:
label_fairness = get_y("Faireness")

In [ ]:
label_loyalty = get_y("In-Group Loyalty")

In [ ]:
label_authority = get_y("Respect For Authority")

In [ ]:
label_purity = get_y("Purity")

## Random Processing

In [ ]:
E1 = get_x(ElPaso, care1, "care")
E1.to_csv("E_feature_E1.csv") 
!cp E_feature_E1.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
E2 = get_x_2(ElPaso, fairness1, "fairness")
E2.to_csv("E_feature_E2.csv") 
!cp E_feature_E2.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
E3 = get_x_2(ElPaso, in_group1, "loyalty")
E3.to_csv("E_feature_E3.csv") 
!cp E_feature_E3.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
E4 = get_x_2(ElPaso, authority1, "authority")
E4.to_csv("E_feature_E4.csv") 
!cp E_feature_E4.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
E5 = get_x_2(ElPaso, purity1, "purity")
E5.to_csv("E_feature_E5.csv") 
!cp E_feature_E5.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
tail = TFIDF.tail(41071)

In [ ]:
tail.reset_index(drop=True, inplace=True)

In [ ]:
tail = tail.drop(["Unnamed: 0"], axis = 1)

In [ ]:
tail.to_csv("tfidf_tail.csv") 
!cp tfidf_tail.csv "/content/drive/MyDrive/DHML/Final Project/"

In [ ]:
E1 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/E_feature_E1.csv")
E2 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/E_feature_E2.csv")
E3 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/E_feature_E3.csv")
E4 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/E_feature_E4.csv")
E5 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/E_feature_E5.csv")
tail = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/tfidf_tail.csv")

In [ ]:
ElPaso_feature = pd.concat([E1, E2, E3, E4, E5, tail], axis = 1)

In [ ]:
ElPaso_feature = ElPaso_feature.drop(["Unnamed: 0"], axis = 1)

In [ ]:
ElPaso_feature.to_csv("ElPaso_feature_whole.csv") 
!cp ElPaso_feature_whole.csv "/content/drive/MyDrive/DHML/Final Project/"

## ElPaso feature

In [ ]:
ElPaso_feature = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/ElPaso_feature_whole.csv")

In [ ]:
ElPaso_feature = ElPaso_feature.drop(["Unnamed: 0"], axis=1)

In [ ]:
ElPaso_care = ElPaso_feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
ElPaso_fairness = ElPaso_feature.drop(["num care", "max_n care", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
ElPaso_loyalty = ElPaso_feature.drop(["num fairness", "max_n fairness", "num care", "max_n care", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
ElPaso_authority = ElPaso_feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num care", "max_n care", "num purity", "max_n purity"], axis = 1)
ElPaso_purity = ElPaso_feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num care", "max_n care"], axis = 1)

## Random Processing

In [ ]:
V1 = get_x(Vegas, care1, "care")
V1.to_csv("V_feature_V1.csv") 
!cp V_feature_V1.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
V2 = get_x_2(Vegas, fairness1, "fairness")
V2.to_csv("V_feature_V2.csv") 
!cp V_feature_V2.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
V3 = get_x_2(Vegas, in_group1, "loyalty")
V3.to_csv("V_feature_V3.csv") 
!cp V_feature_V3.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
V4 = get_x_2(Vegas, authority1, "authority")
V4.to_csv("V_feature_V4.csv") 
!cp V_feature_V4.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
V5 = get_x_2(Vegas, purity1, "purity")
V5.to_csv("V_feature_V5.csv") 
!cp V_feature_V5.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
V1 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/V_feature_V1.csv")
V2 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/V_feature_V2.csv")
V3 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/V_feature_V3.csv")
V4 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/V_feature_V4.csv")
V5 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/V_feature_V5.csv")

In [ ]:
Vegas_feature = pd.concat([V1, V2, V3, V4, V5, TFIDF.head(6205)], axis = 1)

In [ ]:
Vegas_feature = Vegas_feature.drop(["Unnamed: 0"], axis = 1)

In [ ]:
Vegas_feature.to_csv("Vegas_feature_whole.csv") 
!cp Vegas_feature_whole.csv "/content/drive/MyDrive/DHML/Final Project"

## Vegas feature

In [ ]:
Vegas_feature = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_feature_whole.csv")

In [ ]:
Vegas_feature = Vegas_feature.drop(["Unnamed: 0"], axis=1)

In [ ]:
Vegas_care = Vegas_feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
Vegas_fairness = Vegas_feature.drop(["num care", "max_n care", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
Vegas_loyalty = Vegas_feature.drop(["num fairness", "max_n fairness", "num care", "max_n care", "num authority", "max_n authority", "num purity", "max_n purity"], axis = 1)
Vegas_authority = Vegas_feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num care", "max_n care", "num purity", "max_n purity"], axis = 1)
Vegas_purity = Vegas_feature.drop(["num fairness", "max_n fairness", "num loyalty", "max_n loyalty", "num authority", "max_n authority", "num care", "max_n care"], axis = 1)

#Model 1 -- SVM

In [ ]:
def svm_model(feature, label, x_value, c):
  x = feature.to_numpy()
  y = label.to_numpy()
  x_train, x_test, y_train, y_test = train_test_split(x, y,
          test_size=0.20, random_state = 25)
  model_svm = svm.SVC(kernel="linear", C=c)
  model_svm.fit(x_train, y_train)  
  y_predict = model_svm.predict(x_test)
  accuracy = accuracy_score(y_test, y_predict)
  y_predict_data = model_svm.predict(x_value.to_numpy())
  #a, b = model_svm.coef_[0][0], model_svm.coef_[0][1]
  #bias = model_svm.intercept_
  print(accuracy)
  return y_predict_data

C = 1

In [ ]:
svm_model(feature_care, label_care, ElPaso_care.head(), 1)
svm_model(feature_fairness, label_fairness, ElPaso_fairness.head(), 1)
svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty.head(), 1)
svm_model(feature_authority, label_authority, ElPaso_authority.head(), 1)
svm_model(feature_purity, label_purity, ElPaso_purity.head(), 1)

C = 10

In [ ]:
svm_model(feature_care, label_care, ElPaso_care.head(), 10)
svm_model(feature_fairness, label_fairness, ElPaso_fairness.head(), 10)
svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty.head(), 10)
svm_model(feature_authority, label_authority, ElPaso_authority.head(), 10)
svm_model(feature_purity, label_purity, ElPaso_purity.head(), 10)

C = 100

In [ ]:
svm_model(feature_care, label_care, ElPaso_care.head(), 100)
svm_model(feature_fairness, label_fairness, ElPaso_fairness.head(), 100)
svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty.head(), 100)
svm_model(feature_authority, label_authority, ElPaso_authority.head(), 100)
svm_model(feature_purity, label_purity, ElPaso_purity.head(), 100)

C = 1000

In [ ]:
svm_model(feature_care, label_care, ElPaso_care.head(), 1000)
svm_model(feature_fairness, label_fairness, ElPaso_fairness.head(), 1000)
svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty.head(), 1000)
svm_model(feature_authority, label_authority, ElPaso_authority.head(), 1000)
svm_model(feature_purity, label_purity, ElPaso_purity.head(), 1000)

C = 0.1

In [ ]:
svm_model(feature_care, label_care, ElPaso_care.head(), 0.1)
svm_model(feature_fairness, label_fairness, ElPaso_fairness.head(), 0.1)
svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty.head(), 0.1)
svm_model(feature_authority, label_authority, ElPaso_authority.head(), 0.1)
svm_model(feature_purity, label_purity, ElPaso_purity.head(), 0.1)

Run the actual dataset

In [ ]:
# Choose C=10 because C = 1 is really underfitting. It basically estimate 0 for all of them.
ElPaso.insert(3, "care", svm_model(feature_care, label_care, ElPaso_care, 10), True)
ElPaso.insert(4, "fairness", svm_model(feature_fairness, label_fairness, ElPaso_fairness, 10), True)
ElPaso.insert(5, "in_group", svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty, 10), True)
ElPaso.insert(6, "authority", svm_model(feature_authority, label_authority, ElPaso_authority, 10), True)
ElPaso.insert(7, "purity", svm_model(feature_purity, label_purity, ElPaso_purity, 10), True)

In [ ]:
ElPaso.to_csv("El_Paso_model_svm.csv") 
!cp El_Paso_model_svm.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
ElPaso = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/El_Paso_Final.csv")

In [ ]:
ElPaso.insert(3, "care", svm_model(feature_care, label_care, ElPaso_care, 1), True)
ElPaso.insert(4, "fairness", svm_model(feature_fairness, label_fairness, ElPaso_fairness, 1), True)
ElPaso.insert(5, "in_group", svm_model(feature_loyalty, label_loyalty, ElPaso_loyalty, 1), True)
ElPaso.insert(6, "authority", svm_model(feature_authority, label_authority, ElPaso_authority, 1), True)
ElPaso.insert(7, "purity", svm_model(feature_purity, label_purity, ElPaso_purity, 1), True)

In [ ]:
ElPaso.to_csv("El_Paso_model_svm_c_1.csv") 
!cp El_Paso_model_svm_c_1.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
Vegas.insert(3, "care", svm_model(feature_care, label_care, Vegas_care, 10), True)
Vegas.insert(4, "fairness", svm_model(feature_fairness, label_fairness, Vegas_fairness, 10), True)
Vegas.insert(5, "in_group", svm_model(feature_loyalty, label_loyalty, Vegas_loyalty, 10), True)
Vegas.insert(6, "authority", svm_model(feature_authority, label_authority, Vegas_authority, 10), True)
Vegas.insert(7, "purity", svm_model(feature_purity, label_purity, Vegas_purity, 10), True)

In [ ]:
Vegas.to_csv("Vegas_model_svm.csv") 
!cp Vegas_model_svm.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
Vegas = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_Final.csv")

In [ ]:
Vegas.insert(3, "care", svm_model(feature_care, label_care, Vegas_care, 1), True)
Vegas.insert(4, "fairness", svm_model(feature_fairness, label_fairness, Vegas_fairness, 1), True)
Vegas.insert(5, "in_group", svm_model(feature_loyalty, label_loyalty, Vegas_loyalty, 1), True)
Vegas.insert(6, "authority", svm_model(feature_authority, label_authority, Vegas_authority, 1), True)
Vegas.insert(7, "purity", svm_model(feature_purity, label_purity, Vegas_purity, 1), True)

In [ ]:
Vegas.to_csv("Vegas_model_svm.csv") 
!cp Vegas_model_svm.csv "/content/drive/MyDrive/DHML/Final Project"

#Model 2 -- Word2Vec + SVM Model, use moral foundations dictionary 2.0

I am not sure if I want to run this one because it takes forever...

Due to our limited time and limited resource, this model takes a long time to run. Future research could potentially improve on the efficiency of this model and run it on moral foundation dictionary 2.0

In [ ]:
ElPaso2 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/El_Paso_Final.csv")
Vegas2 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_Final.csv")

In [ ]:
care2 = ['compassion', 'empathy', 'kindness', 'caring', 'generosity', 'benevolence', 'altruism', 'compassionate', 'nurture', 'gentleness', 'nurturance', 'sympathy', 'nurturing', 'motherly', 'love', 'beneficence', 'empathize', 'helpfulness', 'loving', 'pity', 'mercy', 'nurturer', 'compassionately', 'nurturers', 'caringly', 'empathising', 'merciful', 'empathizing', 'nurtures', 'warmhearted', 'empathizers', 'protectiveness', 'nurtured', 'benevolent', 'mothering', 'cared', 'healing', 'empathises', 'humane', 'comfort', 'pitied', 'loved', 'altruist', 'cares', 'pitying', 'comforted', 'hug', 'comforting', 'consoling', 'empathizes', 'sympathize', 'care', 'caregiver', 'empathised', 'hugs', 'heal', 'generous', 'condolences', 'mothered', 'charitable', 'generously', 'pities', 'condolence', 'help', 'consolingly', 'solace', 'mother', 'healer', 'hospitality', 'charity', 'empathized', 'healers', 'pityingly', 'mothers', 'child', 'lovingly', 'parenting', 'rescuing', 'rescuer', 'loves', 'consoled', 'clothe', 'sympathizing', 'helping', 'shared', 'childhood', 'mommy', 'vulnerability', 'helpers', 'lover', 'hospitable', 'sharer', 'feeding', 'nursed', 'helper', 'safeness', 'nurses', 'protector', 'motherhood', 'alleviation', 'nursemaid', 'safeguard', 'protect', 'healthiness', 'protecters', 'patient', 'nurse', 'vulnerable', 'benefit', 'feed', 'childcare', 'rescuers', 'hugged', 'helpful', 'rescues', 'nursing', 'protecting', 'heals', 'childbearing', 'hugger', 'relief', 'healed', 'rescued', 'patients', 'share', 'rescue', 'healthy', 'hospitalise', 'hospitalising', 'hugging', 'nursery', 'healthier', 'sharing', 'helps', 'sympathizers', 'hospitalises', 'alleviating', 'wounded', 'wounds', 'hospitalize', 'alleviate', 'protective', 'protection', 'health', 'relieve', 'sympathizer', 'safety', 'beneficiary', 'helped', 'hospital', 'childbirth', 'benefits', 'healthcare', 'relievers', 'feeds', 'hospitalization', 'benefitting', 'relieving', 'safe', 'feeder', 'benefitted', 'hospitalized', 'unharmful', 'protects', 'unharmed', 'protecter', 'safely', 'safekeeping', 'hospitalizing', 'wounding', 'reliever', 'shares', 'relieves', 'alleviates', 'relieved', 'hospitalizes', 'console', 'protectorate', 'alleviated', 'protected', 'wound', 'consoles', 'compassion', 'empathy', 'kindness', 'caring', 'generosity', 'benevolence', 'altruism', 'compassionate', 'nurture', 'gentleness', 'nurturance', 'sympathy', 'nurturing', 'motherly', 'love', 'beneficence', 'empathize', 'helpfulness', 'loving', 'pity', 'mercy', 'nurturer', 'compassionately', 'nurturers', 'caringly', 'empathising', 'merciful', 'empathizing', 'nurtures', 'warmhearted', 'empathizers', 'protectiveness', 'nurtured', 'benevolent', 'mothering', 'cared', 'healing', 'empathises', 'humane', 'comfort', 'pitied', 'loved', 'altruist', 'cares', 'pitying', 'comforted', 'hug', 'comforting', 'consoling', 'empathizes', 'sympathize', 'care', 'caregiver', 'empathised', 'hugs', 'heal', 'generous', 'condolences', 'mothered', 'charitable', 'generously', 'pities', 'condolence', 'help', 'consolingly', 'solace', 'mother', 'healer', 'hospitality', 'charity', 'empathized', 'healers', 'pityingly', 'mothers', 'child', 'lovingly', 'parenting', 'rescuing', 'rescuer', 'loves', 'consoled', 'clothe', 'sympathizing', 'helping', 'shared', 'childhood', 'mommy', 'vulnerability', 'helpers', 'lover', 'hospitable', 'sharer', 'feeding', 'nursed', 'helper', 'safeness', 'nurses', 'protector', 'motherhood', 'alleviation', 'nursemaid', 'safeguard', 'protect', 'healthiness', 'protecters', 'patient', 'nurse', 'vulnerable', 'benefit', 'feed', 'childcare', 'rescuers', 'hugged', 'helpful', 'rescues', 'nursing', 'protecting', 'heals', 'childbearing', 'hugger', 'relief', 'healed', 'rescued', 'patients', 'share', 'rescue', 'healthy', 'hospitalise', 'hospitalising', 'hugging', 'nursery', 'healthier', 'sharing', 'helps', 'sympathizers', 'hospitalises', 'alleviating', 'wounded', 'wounds', 'hospitalize', 'alleviate', 'protective', 'protection', 'health', 'relieve', 'sympathizer', 'safety', 'beneficiary', 'helped', 'hospital', 'childbirth', 'benefits', 'healthcare', 'relievers', 'feeds', 'hospitalization', 'benefitting', 'relieving', 'safe', 'feeder', 'benefitted', 'hospitalized', 'unharmful', 'protects', 'unharmed', 'protecter', 'safely', 'safekeeping', 'hospitalizing', 'wounding', 'reliever', 'shares', 'relieves', 'alleviates', 'relieved', 'hospitalizes', 'console', 'protectorate', 'alleviated', 'protected', 'wound', 'consoles', 'compassion', 'empathy', 'kindness', 'caring', 'generosity', 'benevolence', 'altruism', 'compassionate', 'nurture', 'gentleness', 'nurturance', 'sympathy', 'nurturing', 'motherly', 'love', 'beneficence', 'empathize', 'helpfulness', 'loving', 'pity', 'mercy', 'nurturer', 'compassionately', 'nurturers', 'caringly', 'empathising', 'merciful', 'empathizing', 'nurtures', 'warmhearted', 'empathizers', 'protectiveness', 'nurtured', 'benevolent', 'mothering', 'cared', 'healing', 'empathises', 'humane', 'comfort', 'pitied', 'loved', 'altruist', 'cares', 'pitying', 'comforted', 'hug', 'comforting', 'consoling', 'empathizes', 'sympathize', 'care', 'caregiver', 'empathised', 'hugs', 'heal', 'generous', 'condolences', 'mothered', 'charitable', 'generously', 'pities', 'condolence', 'help', 'consolingly', 'solace', 'mother', 'healer', 'hospitality', 'charity', 'empathized', 'healers', 'pityingly', 'mothers', 'child', 'lovingly', 'parenting', 'rescuing', 'rescuer', 'loves', 'consoled', 'clothe', 'sympathizing', 'helping', 'shared', 'childhood', 'mommy', 'vulnerability', 'helpers', 'lover', 'hospitable', 'sharer', 'feeding', 'nursed', 'helper', 'safeness', 'nurses', 'protector', 'motherhood', 'alleviation', 'nursemaid', 'safeguard', 'protect', 'healthiness', 'protecters', 'patient', 'nurse', 'vulnerable', 'benefit', 'feed', 'childcare', 'rescuers', 'hugged', 'helpful', 'rescues', 'nursing', 'protecting', 'heals', 'childbearing', 'hugger', 'relief', 'healed', 'rescued', 'patients', 'share', 'rescue', 'healthy', 'hospitalise', 'hospitalising', 'hugging', 'nursery', 'healthier', 'sharing', 'helps', 'sympathizers', 'hospitalises', 'alleviating', 'wounded', 'wounds', 'hospitalize', 'alleviate', 'protective', 'protection', 'health', 'relieve', 'sympathizer', 'safety', 'beneficiary', 'helped', 'hospital', 'childbirth', 'benefits', 'healthcare', 'relievers', 'feeds', 'hospitalization', 'benefitting', 'relieving', 'safe', 'feeder', 'benefitted', 'hospitalized', 'unharmful', 'protects', 'unharmed', 'protecter', 'safely', 'safekeeping', 'hospitalizing', 'wounding', 'reliever', 'shares', 'relieves', 'alleviates', 'relieved', 'hospitalizes', 'console', 'protectorate', 'alleviated', 'protected', 'wound', 'consoles', 'compassion', 'empathy', 'kindness', 'caring', 'generosity', 'benevolence', 'altruism', 'compassionate', 'nurture', 'gentleness', 'nurturance', 'sympathy', 'nurturing', 'motherly', 'love', 'beneficence', 'empathize', 'helpfulness', 'loving', 'pity', 'mercy', 'nurturer', 'compassionately', 'nurturers', 'caringly', 'empathising', 'merciful', 'empathizing', 'nurtures', 'warmhearted', 'empathizers', 'protectiveness', 'nurtured', 'benevolent', 'mothering', 'cared', 'healing', 'empathises', 'humane', 'comfort', 'pitied', 'loved', 'altruist', 'cares', 'pitying', 'comforted', 'hug', 'comforting', 'consoling', 'empathizes', 'sympathize', 'care', 'caregiver', 'empathised', 'hugs', 'heal', 'generous', 'condolences', 'mothered', 'charitable', 'generously', 'pities', 'condolence', 'help', 'consolingly', 'solace', 'mother', 'healer', 'hospitality', 'charity', 'empathized', 'healers', 'pityingly', 'mothers', 'child', 'lovingly', 'parenting', 'rescuing', 'rescuer', 'loves', 'consoled', 'clothe', 'sympathizing', 'helping', 'shared', 'childhood', 'mommy', 'vulnerability', 'helpers', 'lover', 'hospitable', 'sharer', 'feeding', 'nursed', 'helper', 'safeness', 'nurses', 'protector', 'motherhood', 'alleviation', 'nursemaid', 'safeguard', 'protect', 'healthiness', 'protecters', 'patient', 'nurse', 'vulnerable', 'benefit', 'feed', 'childcare', 'rescuers', 'hugged', 'helpful', 'rescues', 'nursing', 'protecting', 'heals', 'childbearing', 'hugger', 'relief', 'healed', 'rescued', 'patients', 'share', 'rescue', 'healthy', 'hospitalise', 'hospitalising', 'hugging', 'nursery', 'healthier', 'sharing', 'helps', 'sympathizers', 'hospitalises', 'alleviating', 'wounded', 'wounds', 'hospitalize', 'alleviate', 'protective', 'protection', 'health', 'relieve', 'sympathizer', 'safety', 'beneficiary', 'helped', 'hospital', 'childbirth', 'benefits', 'healthcare', 'relievers', 'feeds', 'hospitalization', 'benefitting', 'relieving', 'safe', 'feeder', 'benefitted', 'hospitalized', 'unharmful', 'protects', 'unharmed', 'protecter', 'safely', 'safekeeping', 'hospitalizing', 'wounding', 'reliever', 'shares', 'relieves', 'alleviates', 'relieved', 'hospitalizes', 'console', 'protectorate', 'alleviated', 'protected', 'wound', 'consoles', 'harm', 'suffer', 'hurt', 'harmed', 'hurting', 'hurts', 'cruel', 'endanger', 'harming', 'harms', 'suffering', 'threaten', 'inflict', 'suffered', 'harmful', 'inflicted', 'mistreat', 'endangers', 'damaging', 'injurious', 'victimize', 'inflicts', 'hurtful', 'suffers', 'inflicting', 'injures', 'vulnerable', 'unkind', 'damage', 'kill', 'die', 'victimizes', 'torment', 'destroy', 'brutalise', 'brutalises', 'distresses', 'endangering', 'mistreats', 'afflict', 'distressing', 'destroys', 'victimises', 'maltreat', 'pain', 'harsh', 'mistreated', 'ravage', 'threatened', 'harass', 'unkindness', 'afflicted', 'threatens', 'threatening', 'distress', 'brutalize', 'tormenting', 'brutalized', 'victimizing', 'damager', 'damaged', 'bully', 'agony', 'abused', 'coldhearted', 'inhuman', 'injured', 'torments', 'brutalizes', 'uncompassionate', 'cruelty', 'tormented', 'mistreating', 'endangered', 'uncaring', 'anguishes', 'destroying', 'killed', 'mistreatment', 'bullied', 'harsher', 'cruelness', 'tortured', 'pained', 'tortures', 'torturing', 'maltreated', 'anguish', 'persecutes', 'maltreatment', 'brutalizing', 'attacked', 'victim', 'crying', 'damages', 'discomforting', 'abusing', 'threat', 'persecute', 'brutalization', 'violent', 'annihilated', 'torturous', 'harasses', 'injurer', 'destroyed', 'molests', 'molest', 'injuring', 'afflicts', 'killing', 'ache', 'wounded', 'persecuted', 'ravages', 'harassed', 'exploited', 'injury', 'brutalisation', 'discomfort', 'unmerciful', 'annihilate', 'exploiters', 'injurers', 'destruction', 'manhandle', 'kills', 'casualties', 'maltreating', 'victims', 'harassing', 'needier', 'smother', 'harassment', 'smothers', 'unhelpful', 'agonize', 'inhumanity', 'duress', 'victimization', 'exploiting', 'cried', 'wounds', 'wounding', 'murderous', 'ravaged', 'uncaringly', 'pains', 'painfulness', 'manhandles', 'bullies', 'assaulted', 'uncharitable', 'distressed', 'persecution', 'murdered', 'ravaging', 'discomforted', 'exploitation', 'torture', 'murderers', 'aches', 'afflictions', 'ungenerous', 'victimizer', 'agonizing', 'paining', 'persecuting', 'exploit', 'harassers', 'malevolent', 'stabs', 'sorrowful', 'assaults', 'needy', 'affliction', 'cries', 'fighting', 'fight', 'attack', 'annihilates', 'sorrow', 'agonized', 'assaulting', 'inhospitable', 'threats', 'ached', 'rapists', 'abuser', 'raped', 'assassinates', 'stabbed', 'inhospitality', 'annihilation', 'punch', 'harshness', 'abusers', 'killer', 'sufferers', 'victimizers', 'smite', 'killers', 'discomforts', 'fatalities', 'molested', 'brutality', 'murdering', 'torturer', 'torturers', 'fights', 'harmfulness', 'bullying', 'casualty', 'sufferer', 'exploiter', 'fatality', 'punches', 'abuses', 'attacks', 'vulnerability', 'carnage', 'tribulation', 'annihilator', 'smothering', 'bullyboy', 'murderer', 'wound', 'stabber', 'tormenters', 'malevolence', 'raping', 'smothered', 'assault', 'genocidal', 'anguishing', 'aching', 'anguished', 'stabbing', 'rapist', 'harasser', 'hungers', 'hunger', 'molesting', 'rape', 'molesters', 'punched', 'violence', 'distressingly', 'molester', 'stabbers', 'neediness', 'assassinate', 'agonizingly', 'tribulations', 'unhelpfulness', 'assaulter', 'puncher', 'punching', 'rapes', 'genocides', 'attackers', 'tormentor', 'assassinations', 'destroyers', 'punchers', 'sorrows', 'tormenter', 'threateningly', 'murder', 'destroyer', 'assassinating', 'crier', 'assassinated', 'molestation', 'attacker', 'murders', 'genocide', 'fighter', 'assassins', 'assaulters', 'hungering', 'achingly', 'hungered', 'murderess', 'assassin', 'exploits', 'fighters', 'assassination']
fairness2 = ['equality', 'fairness', 'justice', 'rights', 'equitable', 'rights', 'fairplay', 'impartiality', 'equal', 'fairminded', 'proportionality', 'equalities', 'fair', 'integrity', 'impartial', 'reciprocity', 'honesty', 'egalitarian', 'right', 'law', 'justness', 'unbias', 'egalitarians', 'parity', 'objectiveness', 'reparations', 'unprejudiced', 'justices', 'laws', 'tribunals', 'retribution', 'reparation', 'lawfully', 'lawful', 'honest', 'compensation', 'lawyers', 'sportsmanship', 'tribunal', 'others', 'rule', 'lawyer', 'proportional', 'equity', 'lawyering', 'trust', 'reciprocal', 'objective', 'justification', 'trustworthiness', 'unbiased', 'vengeance', 'revenge', 'retributions', 'equals', 'equalize', 'refereeing', 'restitution', 'compensating', 'objective', 'back', 'justified', 'justifies', 'retaliation', 'lawyered', 'compensated', 'referees', 'karma', 'share', 'avenger', 'trusting', 'avengers', 'deal', 'trusts', 'compensate', 'trustworthy', 'field', 'tat', 'retaliate', 'field', 'eye', 'deals', 'repayment', 'payback', 'equities', 'justify', 'dues', 'dealing', 'referee', 'repaid', 'dealer', 'equalizer', 'processes', 'fields', 'repay', 'compensates', 'justifying', 'processing', 'process', 'repayments', 'repaying', 'field', 'retaliating', 'dealers', 'retaliated', 'refereed', 'revenger', 'avenging', 'repays', 'trusted', 'avenge', 'retaliates', 'equalizers', 'avenged', 'avenges', 'cheat', 'unfair', 'cheating', 'unfairness', 'injustice', 'fraud', 'dishonest', 'unjust', 'cheated', 'fraudulent', 'cheats', 'frauds', 'dishonesty', 'cheaters', 'deception', 'injustices', 'swindle', 'inequity', 'hypocrisy', 'discrimination', 'unequal', 'cheater', 'inequities', 'defraud', 'racism', 'scam', 'liar', 'defrauds', 'betrayal', 'deceit', 'defrauded', 'inequality', 'liars', 'defrauders', 'hypocrite', 'biased', 'ripoffs', 'scams', 'fleecing', 'defrauder', 'discriminates', 'mislead', 'inequalities', 'prejudice', 'fleeced', 'defrauding', 'ripoff', 'scamming', 'imposters', 'exploitation', 'crooked', 'oppress', 'racist', 'oppression', 'imposter', 'swindled', 'hypocrites', 'plagiarism', 'lied', 'untrustworthiness', 'hoodwink', 'scammed', 'blackmail', 'bilks', 'swindling', 'betrayed', 'bias', 'connive', 'crooks', 'deceive', 'freeloaders', 'favoritism', 'disparity', 'swindles', 'deceived', 'exploiters', 'misleading', 'discriminated', 'bilked', 'deceiving', 'untrustworthy', 'prejudiced', 'advertise', 'scammers', 'swindler', 'theft', 'duplicitous', 'hoodwinked', 'bigoted', 'sexism', 'disproportionate', 'swindlers', 'discriminate', 'conniving', 'sexist', 'betraying', 'hoodwinking', 'partiality', 'misleads', 'disproportion', 'disparity', 'exploiter', 'bilk', 'biases', 'bigots', 'distrust', 'dupe', 'crook', 'racists', 'artist', 'bilking', 'blackmailing', 'deceives', 'betrayers', 'deceiver', 'blackmailed', 'duping', 'shyster', 'connivers', 'imbalanced', 'artists', 'sexists', 'thieving', 'betray', 'imbalance', 'disproportions', 'disproportionately', 'freeloader', 'misleaders', 'connived', 'shysters', 'scammer', 'connives', 'conniver', 'disadvantaged', 'plagiaristic', 'moocher', 'dupes', 'discriminating', 'tricked', 'segregation', 'advertised', 'thief', 'betrayer', 'bigot', 'exploiting', 'lying', 'thieves', 'stealing', 'suckered', 'deceivers', 'bamboozled', 'advertisement', 'freeload', 'bamboozle', 'rob', 'freeloading', 'steal', 'pickpocketing', 'blackmailer', 'prejudicing', 'chauvinists', 'exploit', 'misleader', 'hoodwinks', 'advertiser', 'imbalances', 'pickpocketed', 'exploited', 'pickpockets', 'bamboozles', 'tricking', 'advantage', 'pickpocket', 'advertisers', 'biasing', 'impression', 'bamboozling', 'witness', 'robs', 'moochers', 'betrays', 'robbing', 'advertises', 'impressions', 'blackmails', 'cross', 'blackmailers', 'rob', 'stolen', 'distrustful', 'advertising', 'crossers', 'mooches', 'disproportioned', 'mooching', 'segregated', 'crosser', 'robbed', 'misleadingly', 'segregating', 'stole', 'crosses', 'partial', 'exploits', 'distrusts', 'mooch', 'segregate', 'robbers', 'distrusted', 'crossing', 'distrusting', 'crossed', 'partial', 'on', 'deck', 'robber', 'segregates', 'off', 'trickster', 'off', 'backs', 'mooched', 'deck', 'partial', 'partial', 'deck', 'deck', 'back', 'partial', 'rider', 'off', 'riders', 'deceivingly', 'steals', 'unequaled']
in_group2 = ['player', 'patriot', 'loyal', 'loyalty', 'patriots', 'follower', 'fidelity', 'allegiance', 'ally', 'comrade', 'loyalties', 'part', 'faction', 'comrades', 'allegiances', 'sacrifice', 'allies', 'organization', 'followers', 'them', 'sacrifices', 'one', 'camaraderie', 'comradery', 'all', 'fellow', 'family', 'allegiant', 'corps', 'unity', 'jack', 'uniter', 'glory', 'companions', 'country', 'companion', 'homeland', 'sacrificing', 'indivisible', 'sacrificed', 'solidarity', 'troops', 'nation', 'cult', 'kinship', 'companionship', 'clique', 'allied', 'community', 'group', 'factions', 'familiarity', 'solidarities', 'enlist', 'companionships', 'wife', 'united', 'belongs', 'congregation', 'arms', 'clan', 'trooper', 'sect', 'enlisted', 'enlistment', 'tribalism', 'cohorts', 'war', 'joining', 'troop', 'sacrificial', 'coalition', 'insider', 'pledge', 'cohort', 'enlisting', 'unite', 'communion', 'familiarities', 'belong', 'ingroup', 'belonged', 'company', 'collective', 'fellows', 'cliques', 'uniting', 'clans', 'hazing', 'congregates', 'herd', 'sects', 'uniters', 'undivided', 'unites', 'pledgers', 'coalitions', 'enlists', 'grouping', 'insiders', 'families', 'troupe', 'fellowship', 'kin', 'pledger', 'horde', 'nations', 'tribe', 'hordes', 'pledges', 'herder', 'commune', 'cults', 'congregations', 'organizations', 'herds', 'pledging', 'communities', 'familiar', 'hazings', 'belonging', 'pledged', 'bowed', 'collectively', 'together', 'groups', 'homelands', 'collectives', 'troopers', 'tribes', 'companies', 'countries', 'troupes', 'fellowships', 'tribal', 'communes', 'herders', 'grouped', 'herding', 'congregate', 'herded', 'congregating', 'traitor', 'disloyal', 'treason', 'traitors', 'betray', 'betraying', 'betrayer', 'betrayers', 'unpatriotic', 'betrayed', 'treachery', 'enemies', 'backstabber', 'backstabbed', 'heretic', 'enemy', 'betrays', 'deserter', 'infidels', 'infidel', 'backstab', 'deserting', 'apostate', 'heresy', 'backstabbers', 'heretics', 'unfaithful', 'rebellion', 'desertion', 'deserters', 'apostates', 'unfaithfulness', 'backstabbing', 'rebel', 'on', 'treacherous', 'backstabs', 'heresies', 'outsider', 'outgroup', 'on', 'us', 'on', 'rebels', 'infidelity', 'outgroups', 'rebellions', 'outsiders', 'on']
authority2 = ['respect', 'obey', 'authority', 'obeyed', 'deference', 'reverence', 'respecting', 'obeying', 'tradition', 'adhere', 'obeys', 'revere', 'govern', 'comply', 'respectful', 'honor', 'adhered', 'allegiance', 'dictates', 'nobility', 'forbid', 'dominion', 'governed', 'obedient', 'reveres', 'adhering', 'governs', 'governing', 'oppress', 'respected', 'respectfully', 'honorable', 'dictate', 'commandments', 'commandment', 'venerate', 'politeness', 'respects', 'obedience', 'right', 'forbids', 'permission', 'veneration', 'hierarchy', 'forbade', 'honoring', 'proper', 'venerated', 'stature', 'acquiesce', 'adherence', 'deferential', 'leadership', 'punish', 'forbidding', 'revered', 'piety', 'patriarchs', 'decree', 'coerce', 'dominions', 'dictating', 'venerating', 'crown', 'venerates', 'institution', 'monarchical', 'servant', 'decrees', 'permit', 'says', 'supervise', 'duty', 'compliance', 'lionize', 'supervision', 'orders', 'arms', 'duties', 'dictated', 'elders', 'emperors', 'commands', 'acquiesced', 'emperor', 'adheres', 'servants', 'regulations', 'covenant', 'hierarchical', 'subordinate', 'policing', 'decreeing', 'acquiesces', 'authorizing', 'nobles', 'permits', 'matriarchal', 'authorizes', 'control', 'command', 'subordinating', 'hierarchies', 'reverential', 'deferentially', 'punishes', 'patriarch', 'empires', 'honored', 'allegiant', 'protect', 'traditional', 'subordination', 'punished', 'noble', 'order', 'worship', 'order', 'monarchs', 'ruling', 'example', 'authorities', 'guiding', 'presidents', 'slavishly', 'patriarchy', 'subordinates', 'protection', 'supervisers', 'before', 'fathers', 'down', 'institutions', 'coercion', 'governors', 'commanded', 'police', 'authorize', 'bullys', 'bully', 'protecting', 'acquiescing', 'empire', 'mentor', 'chiefs', 'monarchies', 'honors', 'over', 'acquiescent', 'allegiances', 'bowing', 'oligarchy', 'willing', 'polite', 'supervising', 'order', 'compliantly', 'bishops', 'monarch', 'slaves', 'traitors', 'punishments', 'authorized', 'protector', 'compliant', 'dutiful', 'father', 'punishment', 'coerces', 'line', 'monarchy', 'obediently', 'elder', 'oligarchies', 'dictators', 'leaders', 'bishop', 'over', 'worships', 'coercing', 'protectors', 'dictator', 'protected', 'punishing', 'traitor', 'commanding', 'coerced', 'commanders', 'pope', 'punitive', 'underlings', 'master', 'subordinated', 'president', 'charge', 'matriarchy', 'lionizing', 'slave', 'chief', 'covenants', 'commander', 'matriarch', 'authorizer', 'guide', 'ordered', 'supervised', 'captains', 'punisher', 'supervises', 'bossing', 'commandant', 'governor', 'protects', 'admiral', 'gun', 'bowed', 'dominate', 'arrest', 'mentored', 'ordering', 'submit', 'institutional', 'minister', 'lionizes', 'ranking', 'boss', 'captain', 'book', 'mentors', 'bullies', 'dominant', 'arrested', 'bossed', 'leader', 'rank', 'arresting', 'chieftain', 'ministers', 'regulation', 'superviser', 'dean', 'arrests', 'punishers', 'bullied', 'matriarchs', 'controlling', 'managerial', 'bosses', 'ranks', 'controls', 'dictation', 'guides', 'oligarchs', 'principals', 'dog', 'admirals', 'caste', 'captaining', 'queen', 'mentoring', 'elderly', 'castes', 'governess', 'captained', 'principal', 'bullying', 'submission', 'dominated', 'ladders', 'queens', 'underling', 'ladder', 'fathered', 'dominates', 'dominating', 'presidential', 'oligarch', 'controlled', 'submits', 'submitting', 'honcho', 'commandingly', 'president', 'slaving', 'fathering', 'slaved', 'managers', 'forbiddingly', 'controllers', 'submitted', 'ringleaders', 'ringleader', 'controller', 'ranked', 'manager', 'ministerial', 'submissions', 'ceo', 'punishingly', 'submitter', 'submitters', 'respect', 'obey', 'authority', 'obeyed', 'deference', 'reverence', 'respecting', 'obeying', 'tradition', 'adhere', 'obeys', 'revere', 'govern', 'comply', 'respectful', 'honor', 'adhered', 'allegiance', 'dictates', 'nobility', 'forbid', 'dominion', 'governed', 'obedient', 'reveres', 'adhering', 'governs', 'governing', 'oppress', 'respected', 'respectfully', 'honorable', 'dictate', 'commandments', 'commandment', 'venerate', 'politeness', 'respects', 'obedience', 'right', 'forbids', 'permission', 'veneration', 'hierarchy', 'forbade', 'honoring', 'proper', 'venerated', 'stature', 'acquiesce', 'adherence', 'deferential', 'leadership', 'punish', 'forbidding', 'revered', 'piety', 'patriarchs', 'decree', 'coerce', 'dominions', 'dictating', 'venerating', 'crown', 'venerates', 'institution', 'monarchical', 'servant', 'decrees', 'permit', 'says', 'supervise', 'duty', 'compliance', 'lionize', 'supervision', 'orders', 'arms', 'duties', 'dictated', 'elders', 'emperors', 'commands', 'acquiesced', 'emperor', 'adheres', 'servants', 'regulations', 'covenant', 'hierarchical', 'subordinate', 'policing', 'decreeing', 'acquiesces', 'authorizing', 'nobles', 'permits', 'matriarchal', 'authorizes', 'control', 'command', 'subordinating', 'hierarchies', 'reverential', 'deferentially', 'punishes', 'patriarch', 'empires', 'honored', 'allegiant', 'protect', 'traditional', 'subordination', 'punished', 'noble', 'order', 'worship', 'order', 'monarchs', 'ruling', 'example', 'authorities', 'guiding', 'presidents', 'slavishly', 'patriarchy', 'subordinates', 'protection', 'supervisers', 'before', 'fathers', 'down', 'institutions', 'coercion', 'governors', 'commanded', 'police', 'authorize', 'bullys', 'bully', 'protecting', 'acquiescing', 'empire', 'mentor', 'chiefs', 'monarchies', 'honors', 'over', 'acquiescent', 'allegiances', 'bowing', 'oligarchy', 'willing', 'polite', 'supervising', 'order', 'compliantly', 'bishops', 'monarch', 'slaves', 'traitors', 'punishments', 'authorized', 'protector', 'compliant', 'dutiful', 'father', 'punishment', 'coerces', 'line', 'monarchy', 'obediently', 'elder', 'oligarchies', 'dictators', 'leaders', 'bishop', 'over', 'worships', 'coercing', 'protectors', 'dictator', 'protected', 'punishing', 'traitor', 'commanding', 'coerced', 'commanders', 'pope', 'punitive', 'underlings', 'master', 'subordinated', 'president', 'charge', 'matriarchy', 'lionizing', 'slave', 'chief', 'covenants', 'commander', 'matriarch', 'authorizer', 'guide', 'ordered', 'supervised', 'captains', 'punisher', 'supervises', 'bossing', 'commandant', 'governor', 'protects', 'admiral', 'gun', 'bowed', 'dominate', 'arrest', 'mentored', 'ordering', 'submit', 'institutional', 'minister', 'lionizes', 'ranking', 'boss', 'captain', 'book', 'mentors', 'bullies', 'dominant', 'arrested', 'bossed', 'leader', 'rank', 'arresting', 'chieftain', 'ministers', 'regulation', 'superviser', 'dean', 'arrests', 'punishers', 'bullied', 'matriarchs', 'controlling', 'managerial', 'bosses', 'ranks', 'controls', 'dictation', 'guides', 'oligarchs', 'principals', 'dog', 'admirals', 'caste', 'captaining', 'queen', 'mentoring', 'elderly', 'castes', 'governess', 'captained', 'principal', 'bullying', 'submission', 'dominated', 'ladders', 'queens', 'underling', 'ladder', 'fathered', 'dominates', 'dominating', 'presidential', 'oligarch', 'controlled', 'submits', 'submitting', 'honcho', 'commandingly', 'president', 'slaving', 'fathering', 'slaved', 'managers', 'forbiddingly', 'controllers', 'submitted', 'ringleaders', 'ringleader', 'controller', 'ranked', 'manager', 'ministerial', 'submissions', 'ceo', 'punishingly', 'submitter', 'submitters']
purity2 = ['sanctity', 'sacred', 'sacredness', 'purity', 'wholesome', 'pureness', 'wholesomeness', 'holiness', 'dignity', 'godly', 'piety', 'sanctify', 'chastity', 'undefiled', 'holy', 'sacrosanct', 'pious', 'righteousness', 'dignities', 'sanctified', 'godliness', 'spirituality', 'chaste', 'sanctifies', 'righteous', 'divine', 'religious', 'biblical', 'spiritual', 'deity', 'sanctifying', 'noble', 'modesty', 'decency', 'scriptures', 'nobility', 'religion', 'hallow', 'soul', 'hallowed', 'deism', 'pristine', 'exalted', 'hallowing', 'eternal', 'cross', 'deities', 'faith', 'unadulterated', 'scripture', 'wholesomely', 'divinities', 'worship', 'virgin', 'god', 'catholicism', 'saintly', 'saintliness', 'goddess', 'religiosity', 'purify', 'koranic', 'pure', 'crosses', 'exalt', 'virginity', 'divinity', 'consecrates', 'heaven', 'virginal', 'devout', 'dignified', 'tabernacle', 'exalts', 'buddhas', 'souls', 'temple', 'unsullied', 'heavenly', 'cleanliness', 'abstinance', 'spotlessness', 'talmudic', 'deists', 'gospels', 'prophets', 'religions', 'temples', 'buddhist', 'goddesses', 'saints', 'temperance', 'celibacy', 'consecrated', 'priestly', 'bless', 'marriage', 'prophet', 'exalting', 'unchaste', 'supernatural', 'eternally', 'purification', 'apostles', 'monastic', 'purified', 'communion', 'gods', 'celibate', 'christians', 'theological', 'monasticism', 'unspoiled', 'sterility', 'christian', 'buddha', 'deist', 'prophetic', 'saint', 'righteously', 'apostle', 'prayer', 'faiths', 'shrine', 'purifying', 'worships', 'virgins', 'glorious', 'dignifies', 'atonement', 'deification', 'orthodoxy', 'hallows', 'enshrining', 'nunneries', 'church', 'religiously', 'blessings', 'consecrate', 'gospel', 'pray', 'beatifying', 'yogis', 'theology', 'purifies', 'orthodox', 'untainted', 'torah', 'faithfully', 'catholic', 'heavens', 'yogi', 'consecrating', 'blessed', 'faithful', 'koran', 'abstinence', 'jesus', 'monastery', 'purities', 'consecration', 'catholics', 'prayers', 'prayed', 'sterile', 'blesses', 'enshrined', 'torahs', 'organic', 'bible', 'glory', 'allah', 'glories', 'priests', 'dignifying', 'enshrine', 'mosques', 'spotlessly', 'prude', 'reverend', 'soulful', 'deify', 'christ', 'cathedrals', 'churches', 'cathedral', 'dignify', 'monasteries', 'raw', 'enshrines', 'refinement', 'nuns', 'monks', 'gloriously', 'almighty', 'marring', 'repent', 'prays', 'clean', 'orthodoxies', 'exterminates', 'rabbis', 'spotless', 'bibles', 'mosque', 'immaculate', 'organics', 'purifier', 'foods', 'lord', 'praying', 'repenting', 'marry', 'elevating', 'marrying', 'immaculately', 'rabbi', 'nunnery', 'priest', 'food', 'bloodiness', 'marries', 'synagogues', 'synagogue', 'refined', 'repents', 'angel', 'blessing', 'monk', 'rabbinical', 'organically', 'pope', 'nun', 'nobles', 'prophetically', 'blood', 'repented', 'pastor', 'purifiers', 'lords', 'bloody', 'untouched', 'cleaning', 'exterminating', 'exterminated', 'imam', 'power', 'cleaners', 'married', 'beatification', 'beatify', 'extermination', 'exterminate', 'cleaner', 'body', 'immune', 'atoning', 'imams', 'cleaned', 'atones', 'mary', 'refines', 'cleans', 'atone', 'immunities', 'immunity', 'stainless', 'refining', 'refine', 'atoned', 'exterminator', 'exterminators', 'impurity', 'degradation', 'depravity', 'desecrate', 'desecration', 'repulsiveness', 'degrading', 'decay', 'filth', 'depravities', 'defile', 'sin', 'fornication', 'repulsive', 'depraved', 'impiety', 'degrade', 'repugnance', 'impure', 'degraded', 'desecrations', 'sinfulness', 'impurities', 'indecencies', 'defiled', 'defiles', 'uncleanliness', 'damnation', 'debauchery', 'impious', 'sinful', 'necrophiliacs', 'desecrates', 'sleaziness', 'desecrating', 'desecrated', 'grossness', 'contaminates', 'sinning', 'promiscuity', 'befouls', 'rottenness', 'hedonism', 'revolting', 'repugnant', 'godless', 'scum', 'befoul', 'satanic', 'sluttiness', 'disgusting', 'pestilence', 'debased', 'trashiness', 'sins', 'degradingly', 'corrupting', 'deprave', 'perverted', 'debase', 'fornicating', 'degraders', 'defiling', 'slime', 'horrors', 'repugnantly', 'defiler', 'deviants', 'degrades', 'corrupts', 'debasing', 'perverts', 'parasitic', 'disgusts', 'deflowering', 'hedonistic', 'deviant', 'scummy', 'horrifying', 'necrophilia', 'contamination', 'rot', 'stain', 'contaminating', 'contaminants', 'dirtying', 'debases', 'contaminate', 'abhor', 'heresy', 'sleaze', 'staining', 'defilers', 'harlot', 'plagues', 'sullies', 'fornicators', 'vermin', 'befouling', 'incest', 'trashy', 'excreting', 'deforms', 'abhorred', 'decayed', 'whores', 'deformities', 'perverse', 'adultery', 'fornicate', 'excrement', 'harlots', 'decaying', 'fornicator', 'unclean', 'nauseating', 'sully', 'heresies', 'satan', 'damns', 'satanically', 'sinned', 'sinners', 'adulterous', 'repulses', 'corruption', 'tainting', 'deformity', 'necrophiliac', 'decays', 'corrupted', 'deforming', 'contaminant', 'disgust', 'tarnishes', 'hell', 'filthy', 'taint', 'horrific', 'fecal', 'dirtied', 'flesh', 'stained', 'deform', 'putrid', 'scatalogical', 'dirties', 'whoring', 'cocksucker', 'plague', 'adulterers', 'excretes', 'infesting', 'slimy', 'excrete', 'scuzz', 'horror', 'tarnish', 'sexuality', 'parasite', 'obscenity', 'deformed', 'adulterer', 'befouled', 'muck', 'corpses', 'soiled', 'infest', 'incestuously', 'incestuous', 'fucker', 'devil', 'parasites', 'stains', 'skanks', 'corpse', 'whore', 'lepers', 'curses', 'corrupt', 'pathogens', 'diseased', 'deflower', 'hedonists', 'sinner', 'debaucherous', 'fester', 'hedonist', 'sleazy', 'fucks', 'promiscuous', 'cursed', 'curse', 'apostates', 'cocksuckers', 'heretic', 'lewdness', 'slutty', 'infests', 'festers', 'pervert', 'fuck', 'skanky', 'dirty', 'mucky', 'puke', 'alcoholism', 'feces', 'sullied', 'disgustingly', 'sexual', 'cunt', 'taints', 'profane', 'heretics', 'fucking', 'tarnishing', 'fornicated', 'mar', 'shitting', 'slut', 'obscene', 'barf', 'rotten', 'disgusted', 'cunts', 'waste', 'parasitically', 'sinfully', 'wastes', 'vomit', 'pathogen', 'rats', 'pathogenic', 'indecent', 'infect', 'leper', 'indecently', 'shit', 'abhors', 'skank', 'infestation', 'deflowered', 'leprosy', 'diseases', 'heretical', 'dirt', 'cursing', 'tarnishment', 'disease', 'prostitution', 'infested', 'apostate', 'sluts', 'fuckers', 'profanity', 'addiction', 'contaminated', 'scuzzy', 'infectiousness', 'indecency', 'vomited', 'germ', 'prostituting', 'excreted', 'rubbish', 'fucked', 'sodomy', 'untouchables', 'epidemics', 'swear', 'shits', 'whorehouses', 'pigsty', 'germs', 'prostituted', 'mud', 'dung', 'epidemic', 'rat', 'douchebag', 'perversely', 'pukes', 'puking', 'prostitutes', 'barfs', 'slutting', 'trashing', 'whored', 'douchebags', 'infection', 'shite', 'spoil', 'gross', 'repulsed', 'pus', 'festering', 'cockroaches', 'tainted', 'contagion', 'barfed', 'infects', 'damned', 'addictions', 'shitty', 'skanking', 'trash', 'whorehouse', 'phlegm', 'moldy', 'plaguing', 'shat', 'drugged', 'garbage', 'infecting', 'pandemics', 'viruses', 'nauseated', 'cockroach', 'puked', 'drugging', 'manure', 'mucking', 'lewd', 'alcoholics', 'gangrenous', 'barfing', 'gangrene', 'shitter', 'shittier', 'tarnished', 'cock', 'vomits', 'hookers', 'damn', 'addict', 'alcoholic', 'nausea', 'swearing', 'vomiting', 'skanked', 'infections', 'foul', 'prostitute', 'risque', 'lice', 'gonorrhea', 'wasting', 'profanities', 'divorces', 'crappy', 'spreading', 'wasters', 'addicting', 'trashed', 'addicts', 'scabies', 'swore', 'nauseous', 'phlegmatically', 'spoiling', 'nauseatingly', 'drugs', 'virus', 'waster', 'untouchable', 'addicted', 'damning', 'pandemic', 'hooker', 'bm', 'infected', 'festered', 'marred', 'phlegmatic', 'divorce', 'viral', 'contagiously', 'plagued', 'repulsing', 'swears', 'drug', 'spoiled', 'cum', 'divorcing', 'wasted', 'divorced', 'contagious']

In [ ]:
ElPaso2.insert(3, "care", classification1(ElPaso2, care2, "care"), True)
ElPaso2.insert(4, "fairness", classification1(ElPaso2, fairness2, "fairness"), True)
ElPaso2.insert(5, "in_group", classification1(ElPaso2, in_group2, "in_group"), True)
ElPaso2.insert(6, "authority", classification1(ElPaso2, authority2, "authority"), True)
ElPaso2.insert(7, "purity", classification1(ElPaso2, purity2, "purity"), True)

In [ ]:
ElPaso2.to_csv("El_Paso_model2.csv") 
!cp El_Paso_model2.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
Vegas2.insert(3, "care", classification1(Vegas2, care2, "care"), True)
Vegas2.insert(4, "fairness", classification1(Vegas2, fairness2, "fairness"), True)
Vegas2.insert(5, "in_group", classification1(Vegas2, in_group2, "in_group"), True)
Vegas2.insert(6, "authority", classification1(Vegas2, authority2, "authority"), True)
Vegas2.insert(7, "purity", classification1(Vegas2, purity2, "purity"), True)

In [ ]:
Vegas2.to_csv("Vegas_model2.csv") 
!cp Vegas2_model2.csv "/content/drive/MyDrive/DHML/Final Project"

#Model 3 -- Word2Vec + KNN

In [ ]:
ElPaso3 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/El_Paso_Final.csv")
Vegas3 = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_Final.csv")

In [ ]:
def knn_model(feature, label, x_value):
  x = feature.to_numpy()
  #print(x)
  y = label.to_numpy()
  #print(y)
  X_train, X_test, Y_train, Y_test = train_test_split(x, y,
        test_size=0.20, random_state = 25)
  accuracy_list = []
  y_predict_list = []
  for nn_neighbors in range(30):
    if nn_neighbors == 0:
      continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    #Y_difference = abs(Y_test - Y_pred)
    #accuracy = float(len(Y_difference[Y_difference == 0])) / len(Y_difference)
    print("Accuracy (%) for kNN with ", nn_neighbors, " nearest neighbors is: ", round(accuracy * 100,2))
    accuracy_list.append(accuracy)
    y_predict_data = knn.predict(x_value)
    print(y_predict_data)
    #y_predict_list.append(y_predict_data)
  #max_index = accuracy_list.index(max(accuracy_list))
  #print("num of k: "+ str(max_index+1)) 
  #print("accuracy: "+ str(max(accuracy_list)))
  #return y_predict_list[29]

Since the original knn model takes a very long time, I write one for each of the categories

In [ ]:
def knn_model_care(feature, label, x_value):
  x = feature.to_numpy()
  #print(x)
  y = label.to_numpy()
  #print(y)
  X_train, X_test, Y_train, Y_test = train_test_split(x, y,
        test_size=0.20, random_state = 25)
  for nn_neighbors in range(30):
    if nn_neighbors == 0:
      continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    if nn_neighbors == 28:
      y_predict_data = knn.predict(x_value)
      print(accuracy)
      return y_predict_data

In [ ]:
def knn_model_fairness(feature, label, x_value):
  x = feature.to_numpy()
  #print(x)
  y = label.to_numpy()
  #print(y)
  X_train, X_test, Y_train, Y_test = train_test_split(x, y,
        test_size=0.20, random_state = 25)
  for nn_neighbors in range(30):
    if nn_neighbors == 0:
      continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    if nn_neighbors == 7:
      y_predict_data = knn.predict(x_value)
      print(accuracy)
      return y_predict_data

In [ ]:
def knn_model_loyalty(feature, label, x_value):
  x = feature.to_numpy()
  #print(x)
  y = label.to_numpy()
  #print(y)
  X_train, X_test, Y_train, Y_test = train_test_split(x, y,
        test_size=0.20, random_state = 25)
  for nn_neighbors in range(30):
    if nn_neighbors == 0:
      continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    if nn_neighbors == 10:
      y_predict_data = knn.predict(x_value)
      print(accuracy)
      return y_predict_data

In [ ]:
def knn_model_authority(feature, label, x_value):
  x = feature.to_numpy()
  #print(x)
  y = label.to_numpy()
  #print(y)
  X_train, X_test, Y_train, Y_test = train_test_split(x, y,
        test_size=0.20, random_state = 25)
  for nn_neighbors in range(30):
    if nn_neighbors == 0:
      continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    if nn_neighbors == 7:
      y_predict_data = knn.predict(x_value)
      print(accuracy)
      return y_predict_data

In [ ]:
def knn_model_purity(feature, label, x_value):
  x = feature.to_numpy()
  #print(x)
  y = label.to_numpy()
  #print(y)
  X_train, X_test, Y_train, Y_test = train_test_split(x, y,
        test_size=0.20, random_state = 25)
  for nn_neighbors in range(30):
    if nn_neighbors == 0:
      continue
    knn = KNeighborsClassifier(n_neighbors=nn_neighbors)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    if nn_neighbors == 16:
      y_predict_data = knn.predict(x_value)
      print(accuracy)
      return y_predict_data

In [ ]:
ElPaso3.insert(3, "care", knn_model_care(feature_care, label_care, ElPaso_care), True)
ElPaso3.insert(4, "fairness", knn_model_fairness(feature_fairness, label_fairness, ElPaso_fairness), True)
ElPaso3.insert(5, "in_group", knn_model_loyalty(feature_loyalty, label_loyalty, ElPaso_loyalty), True)
ElPaso3.insert(6, "authority", knn_model_authority(feature_authority, label_authority, ElPaso_authority), True)
ElPaso3.insert(7, "purity", knn_model_purity(feature_purity, label_purity, ElPaso_purity), True)

In [ ]:
ElPaso3.to_csv("El_Paso_model_knn.csv") 
!cp El_Paso_model_knn.csv "/content/drive/MyDrive/DHML/Final Project"

In [ ]:
Vegas3.insert(3, "care", knn_model_care(feature_care, label_care, Vegas_care), True)
Vegas3.insert(4, "fairness", knn_model_fairness(feature_fairness, label_fairness, Vegas_fairness), True)
Vegas3.insert(5, "in_group", knn_model_loyalty(feature_loyalty, label_loyalty, Vegas_loyalty), True)
Vegas3.insert(6, "authority", knn_model_authority(feature_authority, label_authority, Vegas_authority), True)
Vegas3.insert(7, "purity", knn_model_purity(feature_purity, label_purity, Vegas_purity), True)

In [ ]:
Vegas3.to_csv("Vegas_model_knn.csv") 
!cp Vegas_model_knn.csv "/content/drive/MyDrive/DHML/Final Project"

#Discussion

Some processing before the discussion

In [ ]:
ElPaso_data = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/El_Paso_model_svm.csv") #choose the model with the best accuracy
Vegas_data = pd.read_csv("/content/drive/MyDrive/DHML/Final Project/Vegas_model_svm.csv")

In [ ]:
ElPaso_data = ElPaso_data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1)

In [ ]:
Vegas_data = Vegas_data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1)


Split the data by time after the attack: 18h

El Paso attack happens in 2019.08.03, 6:09 UTC - 6:15 UTC

Vegas attack happens in 2017.10.1 5:05 am UTC

In [ ]:
def time_split(data, time1, time2, before, after):
  for index, row in tqdm(enumerate(data.values)):
    date = data["created_at"][index]
    #if date < time1:
      #data = data.drop([index])
    if index in data["created_at"]:
      if  str(date) < time2:
        before.append(row)
      else:
        after.append(row)
  before = pd.DataFrame(before, columns=["created_at", "text", "care", "fairness", "in_group", "purity", "authority"])
  after = pd.DataFrame(after, columns=["created_at", "text", "care", "fairness", "in_group", "purity", "authority"])
  return before, after

In [ ]:
before18h_elpaso = []
after18h_elpaso = []

before18h_elpaso, after18h_elpaso = time_split(ElPaso_data, "2019-08-03T06:09:00", "2019-08-04T00:09:00", before18h_elpaso, after18h_elpaso)

In [ ]:
before18h_vegas = []
after18h_vegas = []

before18h_vegas, after18h_vegas = time_split(Vegas_data, "2017-10-02T05:05:00", "2017-10-02T23:05:00", before18h_vegas, after18h_vegas)

Analyze 

In [ ]:
def count (data, category):
  count = 0
  for num in data[category]:
    try:
      count += int(num)
    except:
      continue
  
  return count

El Paso

In [ ]:
num_before18h_elpaso = len(before18h_elpaso)
num_after18h_elpaso = len(after18h_elpaso)

In [ ]:
print(count(before18h_elpaso, "care")/num_before18h_elpaso)
print(count(after18h_elpaso, "care")/num_after18h_elpaso )

0.38365462937822425
0.4999558615819209


In [ ]:
print(count(before18h_elpaso, "fairness")/num_before18h_elpaso)
print(count(after18h_elpaso, "fairness")/num_after18h_elpaso )

0.1849579147434157
0.2597987288135593


In [ ]:
print(count(before18h_elpaso, "in_group")/num_before18h_elpaso)
print(count(after18h_elpaso, "in_group")/num_after18h_elpaso )

0.04887320119467825
0.08421610169491525


In [ ]:
print(count(before18h_elpaso, "purity")/num_before18h_elpaso)
print(count(after18h_elpaso, "purity")/num_after18h_elpaso )

0.1029052402932392
0.06077860169491525


In [ ]:
print(count(before18h_elpaso, "authority")/num_before18h_elpaso)
print(count(after18h_elpaso, "authority")/num_after18h_elpaso )

0.24349714906326364
0.3095868644067797


Vegas

In [ ]:
num_before18h_vegas = len(before18h_vegas)
num_after18h_vegas = len(after18h_vegas)

In [ ]:
print(count(before18h_vegas, "care")/num_before18h_vegas)
print(count(after18h_vegas, "care")/num_after18h_vegas)

0.5284414106939704
0.4220899962811454


In [ ]:
print(count(before18h_vegas, "fairness")/num_before18h_vegas)
print(count(after18h_vegas, "fairness")/num_after18h_vegas)

0.1541524459613197
0.13536630717738937


In [ ]:
print(count(before18h_vegas, "in_group")/num_before18h_vegas)
print(count(after18h_vegas, "in_group")/num_after18h_vegas)

0.06228668941979522
0.03495723317218297


In [ ]:
print(count(before18h_vegas, "purity")/num_before18h_vegas)
print(count(after18h_vegas, "purity")/num_after18h_vegas)

0.04436860068259386
0.03495723317218297


In [ ]:
print(count(before18h_vegas, "authority")/num_before18h_vegas)
print(count(after18h_vegas, "authority")/num_after18h_vegas)

0.19823663253697382
0.12867236891037562
